In [4]:
import pandas as pd
import duckdb

In [5]:
people = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [24, 25, 26],
    'city': ['New York', 'San Francisco', 'Los Angeles']
})

In [7]:
duckdb.sql("from people")

┌─────────┬───────┬───────────────┐
│  name   │  age  │     city      │
│ varchar │ int64 │    varchar    │
├─────────┼───────┼───────────────┤
│ Alice   │    24 │ New York      │
│ Bob     │    25 │ San Francisco │
│ Charlie │    26 │ Los Angeles   │
└─────────┴───────┴───────────────┘

In [11]:
params = {"city": "San Francisco"}
sf_df = duckdb.execute("""select * from people where city = $city""", params).fetchdf()

In [12]:
no_sf_df = duckdb.sql("from people").filter("city <> 'San Francisco'").fetchdf()

In [29]:
#User defined functions
def add_one(x:int) -> int:
    return x + 1

def remove_spaces(field:str) -> str:
    if field:
        return field.lstrip().rstrip()
    else:
        return field
    
def add_michal(greeting:str) -> str:
    return greeting + " Michal!"

In [19]:
con = duckdb.connect(database=':memory:')

In [31]:
#Register with duckdb

#con.create_function('add_one', add_one)
#con.create_function("remove_spaces", remove_spaces)
con.create_function("add_michal", add_michal)

In [32]:
con.sql("""SELECT function_name, function_type, parameters, parameter_types, return_type from duckdb_functions() where function_name = 'add_michal'""")

┌───────────────┬───────────────┬────────────┬─────────────────┬─────────────┐
│ function_name │ function_type │ parameters │ parameter_types │ return_type │
│    varchar    │    varchar    │ varchar[]  │    varchar[]    │   varchar   │
├───────────────┼───────────────┼────────────┼─────────────────┼─────────────┤
│ add_michal    │ scalar        │ [col0]     │ [VARCHAR]       │ VARCHAR     │
└───────────────┴───────────────┴────────────┴─────────────────┴─────────────┘

In [27]:
con.sql("select length(remove_spaces('    foo '))")

┌───────────────────────────────────┐
│ length(remove_spaces('    foo ')) │
│               int64               │
├───────────────────────────────────┤
│                                 3 │
└───────────────────────────────────┘

In [28]:
con.sql("select add_one(2)")

┌────────────┐
│ add_one(2) │
│   int64    │
├────────────┤
│          3 │
└────────────┘

In [33]:
con.sql("select add_michal('Yo')")

┌──────────────────┐
│ add_michal('Yo') │
│     varchar      │
├──────────────────┤
│ Yo Michal!       │
└──────────────────┘

In [36]:
import polars
con.execute("INSTALL httpfs")
con.execute("LOAD httpfs")

In [37]:
population = con.read_csv("https://bit.ly/3KoiZR0")

In [38]:
population_table = con.table("population")

Convert relation to Polars

In [39]:
population_table.limit(5).pl()

Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
str,str,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str
"""Afghanistan ""","""ASIA (EX. NEAR EAST) """,31056997,647500,"""48,0""","""0,00""","""23,06""","""163,07""",700,"""36,0""","""3,2""","""12,13""","""0,22""","""87,65""","""1""","""46,6""","""20,34""","""0,38""","""0,24""","""0,38"""
"""Albania ""","""EASTERN EUROPE …",3581655,28748,"""124,6""","""1,26""","""-4,93""","""21,52""",4500,"""86,5""","""71,2""","""21,09""","""4,42""","""74,49""","""3""","""15,11""","""5,22""","""0,232""","""0,188""","""0,579"""
"""Algeria ""","""NORTHERN AFRICA …",32930091,2381740,"""13,8""","""0,04""","""-0,39""","""31""",6000,"""70,0""","""78,1""","""3,22""","""0,25""","""96,53""","""1""","""17,14""","""4,61""","""0,101""","""0,6""","""0,298"""
"""American Samoa ""","""OCEANIA …",57794,199,"""290,4""","""58,29""","""-20,71""","""9,27""",8000,"""97,0""","""259,5""","""10""","""15""","""75""","""2""","""22,46""","""3,27""",null,null,null
"""Andorra ""","""WESTERN EUROPE …",71201,468,"""152,1""","""0,00""","""6,6""","""4,05""",19000,"""100,0""","""497,2""","""2,22""","""0""","""97,78""","""3""","""8,71""","""6,25""",null,null,null


In [40]:
arrow_table = population_table.to_arrow_table()